In [1]:
from sklearn import datasets 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import os  
import math
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import h5py
from PIL import Image
import numpy as np
import os

# 카테고리 지정하기
categories = ["Good","Normal","Bad","Very_Bad"]
nb_classes = len(categories)
# 이미지 크기 지정하기
image_w = int(1280/4)
image_h = int(720/4)

# 데이터 불러오기 --- (※1)
x_train, x_test, y_train, y_test = np.load("./image/5obj.npy")

# 데이터 정규화하기
x_train = x_train.astype("float") / 256
x_test  = x_test.astype("float")  / 256

/usr/local/lib/python3.6/dist-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras import optimizers
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

model = Sequential()
# 모델 구축하기 --- (※2 CNN)
model.add(Convolution2D(32, 3, 3, border_mode='same',input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten()) # --- (※3) 
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

# 모델 훈련하기 --- (※4)
hdf5_file = "./image/model1_sgd_epoch100.hdf5"
if os.path.exists(hdf5_file):
    model.load_weights(hdf5_file)
else:
    model.fit(x_train, y_train, batch_size=128, epochs=100)
    
#model.save_weights(hdf5_file) 
#model.save_weights(hdf5_file) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(180, 320,..., padding="same")`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same")`
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  


In [26]:
# 모델 평가하기--- (※5)

result = pd.DataFrame(index=['Very_Bad','Bad','Normal','Good'],columns=['Very_Bad','Bad','Normal','Good'])
result[:] = 0
result

pre = model.predict(x_test)
for i,v in enumerate(pre):
    pre_ans = v.argmax()
    ans = y_test[i].argmax()
    dat = x_test[i]
    result[categories[ans]][categories[pre_ans]] = result[categories[ans]][categories[pre_ans]] + 1
    if ans == pre_ans: continue
    print("[NG]", categories[pre_ans], "!=", categories[ans])
    print(v)
    fname = "image/error/" + str(i) + "-" + categories[pre_ans] + \
        "-ne-" + categories[ans] + ".png"
    dat *= 256
    img = Image.fromarray(np.uint8(dat))
    img.save(fname)

score = model.evaluate(x_test, y_test)
print('loss=', score[0])
print('accuracy=', score[1])


[NG] Bad != Very_Bad
[0. 0. 1. 0.]
[NG] Bad != Normal
[0. 0. 1. 0.]
[NG] Bad != Very_Bad
[0. 0. 1. 0.]
[NG] Bad != Normal
[0. 0. 1. 0.]
[NG] Good != Normal
[1. 0. 0. 0.]
[NG] Normal != Good
[0. 1. 0. 0.]
[NG] Bad != Normal
[0. 0. 1. 0.]
[NG] Normal != Bad
[0. 1. 0. 0.]
[NG] Bad != Normal
[0. 0. 1. 0.]
[NG] Normal != Good
[0. 1. 0. 0.]
[NG] Normal != Good
[0. 1. 0. 0.]
[NG] Normal != Bad
[0. 1. 0. 0.]
[NG] Bad != Normal
[0. 0. 1. 0.]
[NG] Normal != Bad
[0. 1. 0. 0.]
[NG] Normal != Bad
[0. 1. 0. 0.]
[NG] Normal != Bad
[0. 1. 0. 0.]
[NG] Bad != Normal
[0. 0. 1. 0.]
[NG] Bad != Normal
[0. 0. 1. 0.]
[NG] Good != Normal
[1. 0. 0. 0.]
[NG] Bad != Good
[0. 0. 1. 0.]
[NG] Bad != Normal
[0. 0. 1. 0.]
[NG] Bad != Normal
[0. 0. 1. 0.]
[NG] Normal != Bad
[0. 1. 0. 0.]
[NG] Bad != Normal
[0. 0. 1. 0.]
[NG] Bad != Very_Bad
[0. 0. 1. 0.]
[NG] Normal != Bad
[0. 1. 0. 0.]
[NG] Normal != Good
[0. 1. 0. 0.]
[NG] Bad != Good
[0. 0. 1. 0.]
[NG] Bad != Very_Bad
[0. 0. 1. 0.]
[NG] Normal != Good
[0. 1. 0. 0.]

[NG] Normal != Good
[0. 1. 0. 0.]
[NG] Normal != Good
[0. 1. 0. 0.]
[NG] Bad != Normal
[0. 0. 1. 0.]
[NG] Normal != Good
[0. 1. 0. 0.]
[NG] Normal != Very_Bad
[0. 1. 0. 0.]
[NG] Bad != Normal
[0. 0. 1. 0.]
[NG] Normal != Bad
[0. 1. 0. 0.]
[NG] Good != Normal
[1. 0. 0. 0.]
[NG] Good != Bad
[1. 0. 0. 0.]
[NG] Good != Normal
[1. 0. 0. 0.]
[NG] Normal != Good
[0. 1. 0. 0.]
[NG] Bad != Normal
[0. 0. 1. 0.]
[NG] Bad != Normal
[0. 0. 1. 0.]
[NG] Normal != Bad
[0. 1. 0. 0.]
[NG] Bad != Good
[0. 0. 1. 0.]
[NG] Normal != Bad
[0. 1. 0. 0.]
[NG] Bad != Normal
[0. 0. 1. 0.]
[NG] Bad != Very_Bad
[0. 0. 1. 0.]
[NG] Normal != Good
[0. 1. 0. 0.]
[NG] Good != Normal
[1. 0. 0. 0.]
[NG] Good != Normal
[1. 0. 0. 0.]
[NG] Bad != Normal
[0. 0. 1. 0.]
[NG] Normal != Bad
[0. 1. 0. 0.]
[NG] Bad != Very_Bad
[0. 0. 1. 0.]
[NG] Normal != Bad
[0. 1. 0. 0.]
[NG] Bad != Normal
[0. 0. 1. 0.]
[NG] Bad != Normal
[0. 0. 1. 0.]
[NG] Normal != Bad
[0. 1. 0. 0.]
[NG] Bad != Normal
[0. 0. 1. 0.]
[NG] Bad != Normal
[0. 0. 1. 